# Básico de SQLite

Seguindo o tutorial de https://www.youtube.com/watch?v=byHcYRpMgI4

In [5]:
# Importando sqlite
import sqlite3

In [6]:
# Criando ou nos conectando a um banco de dados
conexao = sqlite3.connect("clientes.db")
# Caso queiramos criar um banco de dados momentâneo em memória
# conexao = sqlite3.connect(":memory:")

In [7]:
# Criando cursor que nos permite interagir com o banco de dados
c = conexao.cursor()

In [8]:
# Criando uma tabela, lembrando que sqlite é case sensitive
# Tipos de dados do sqlite: NULL, INTEGER, REAL, TEXT, BLOB
c.execute("""
        CREATE TABLE clientes(
        primeiro_nome TEXT,
        ultimo_nome TEXT,
        email TEXT
        )
""")

In [9]:
## Adicionando dados a nossa tabela
c.execute("""
    INSERT INTO clientes 
    VALUES ("John","Textor", "glorioso_2022@botafogo.com.br")
""")

In [10]:
# Ao invés de adicionarmos um registro por vez podemos usar listas de python para adicionar vários registros
varios_clientes = [
    ("nome_1", "sobrenome_1", "nome_1@sobrenome_1"),
    ("nome_2", "sobrenome_2", "nome_2@sobrenome_2"),
    ("nome_3", "sobrenome_3", "nome_3@sobrenome_3"),
]
c.executemany("""
    INSERT INTO clientes 
    VALUES (?,?,?)
""", varios_clientes )

In [11]:
# Realizando query sobre tabela criada
c.execute("SELECT * FROM clientes")

In [12]:
# Query que retorna apenas a primeira linha da tabela: c.fetchone()
# Query que retorna apenas as três primeiras linhas da tabela: c.fetchmany(3)
# Query que retorna todas as linhas da tabela
# O retorno da query é feita através de uma lista python
print(c.fetchall())

[('John', 'Textor', 'glorioso_2022@botafogo.com.br'), ('nome_1', 'sobrenome_1', 'nome_1@sobrenome_1'), ('nome_2', 'sobrenome_2', 'nome_2@sobrenome_2'), ('nome_3', 'sobrenome_3', 'nome_3@sobrenome_3')]


In [13]:
# Por baixo dos panos o sqlite define uma chave primária chamada rowid
c.execute("SELECT rowid, * FROM clientes")
print(c.fetchall())

[(1, 'John', 'Textor', 'glorioso_2022@botafogo.com.br'), (2, 'nome_1', 'sobrenome_1', 'nome_1@sobrenome_1'), (3, 'nome_2', 'sobrenome_2', 'nome_2@sobrenome_2'), (4, 'nome_3', 'sobrenome_3', 'nome_3@sobrenome_3')]


In [14]:
# Criando uma cláusula com WHERE
c.execute("SELECT rowid, * FROM clientes WHERE primeiro_nome='John' ")
print(c.fetchall())

[(1, 'John', 'Textor', 'glorioso_2022@botafogo.com.br')]


In [15]:
# Commitando os comandos associados ao cursor através da nossa conexao
conexao.commit()

In [16]:
# Fechando a conexão
conexao.close()

# Conversão de .db para dataframe

Seguindo o tutorial de https://datatofish.com/sql-to-pandas-dataframe/

In [17]:
# Importando pacote e criando conector e cursor
import sqlite3
conn = sqlite3.connect('produtos.db') 
c = conn.cursor()

In [18]:
# Criando tabela de produtos
c.execute('''
          CREATE TABLE IF NOT EXISTS produtos
          (produto_id INTEGER PRIMARY KEY, produto_nome TEXT, preco INTEGER)
          ''')

In [20]:
# Inserindo dados na tabela
c.execute('''
          INSERT INTO produtos (produto_id, produto_nome, preco)

                VALUES
                (1,'Computer',800),
                (2,'Printer',200),
                (3,'Tablet',300),
                (4,'Desk',450),
                (5,'Chair',150)
          ''')    

In [22]:
import pandas as pd

In [23]:
# Comitando alterações do cursor
conn.commit()

In [24]:
# Criando novo cursor para usar com a biblioteca pandas
conn = sqlite3.connect('produtos.db') 

In [25]:
# Passando os dados de um db do sqlite para um dataframe pandas
dataframe_sqlite = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM produtos
                               ''', conn)

In [26]:
# Visualizando o dataframe que transformamos
dataframe_sqlite

,produto_id,produto_nome,preco
0,1,Computer,800
1,2,Printer,200
2,3,Tablet,300
3,4,Desk,450
4,5,Chair,150


# Conversão de dataframe para .db

In [27]:
# Instalando SQLAlchemy, toolkit que permite ao Python conversar com DBs SQL
!pip install sqlalchemy

     |████████████████████████████████| 1.6 MB 3.6 MB/s eta 0:00:01


In [28]:
# Importando read_csv para ler tabela que vamos passar para o .db
from pandas import read_csv

In [30]:
# Lendo dados de .csv para dataframe
df_brasil = read_csv("brasil.csv")

In [31]:
# Verificando o dataframe
df_brasil.head()

,ano,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5,nivel_6,tipo_emissao,gas,atividade_economica,produto,emissao
0,1970,Agropecuária,Cultivo do Arroz,Diretas,Outros,Vegetal,Arroz,Emissão,CH4 (t),NaN,NaN,230462.17
1,1971,Agropecuária,Cultivo do Arroz,Diretas,Outros,Vegetal,Arroz,Emissão,CH4 (t),NaN,NaN,226016.30
2,1972,Agropecuária,Cultivo do Arroz,Diretas,Outros,Vegetal,Arroz,Emissão,CH4 (t),NaN,NaN,220101.20
3,1973,Agropecuária,Cultivo do Arroz,Diretas,Outros,Vegetal,Arroz,Emissão,CH4 (t),NaN,NaN,214195.56
4,1974,Agropecuária,Cultivo do Arroz,Diretas,Outros,Vegetal,Arroz,Emissão,CH4 (t),NaN,NaN,186862.84


In [32]:
# Importando função que fará conexão com .db do SQLAlchemy
from sqlalchemy import create_engine

In [33]:
# Criando engine e conector com o .db do sqlite, o echo é para vermos a conexão como log
engine = create_engine('sqlite:///pandas_df.db', echo=True)
sqlite_connection = engine.connect()

In [34]:
# Convertendo nosso df para um .db
df_brasil.to_sql("brasil_emissao", sqlite_connection, if_exists='fail')

2022-04-17 22:26:48,091 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("brasil_emissao")
2022-04-17 22:26:48,092 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-17 22:26:48,094 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("brasil_emissao")
2022-04-17 22:26:48,094 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-17 22:26:48,098 INFO sqlalchemy.engine.Engine 
CREATE TABLE brasil_emissao (
	"index" BIGINT, 
	ano BIGINT, 
	nivel_1 TEXT, 
	nivel_2 TEXT, 
	nivel_3 TEXT, 
	nivel_4 TEXT, 
	nivel_5 TEXT, 
	nivel_6 TEXT, 
	tipo_emissao TEXT, 
	gas TEXT, 
	atividade_economica TEXT, 
	produto TEXT, 
	emissao FLOAT
)


2022-04-17 22:26:48,100 INFO sqlalchemy.engine.Engine [no key 0.00155s] ()
2022-04-17 22:26:48,501 INFO sqlalchemy.engine.Engine COMMIT
2022-04-17 22:26:48,503 INFO sqlalchemy.engine.Engine CREATE INDEX ix_brasil_emissao_index ON brasil_emissao ("index")
2022-04-17 22:26:48,504 INFO sqlalchemy.engine.Engine [no key 0.00115s] ()
2022-04-17 22:26:48,716 INFO sqlalche

454850

In [35]:
# Fazendo um query sobre o nosso .db para confirmar se deu tudo certo
conexao = sqlite3.connect("pandas_df.db")
c = conexao.cursor()
c.execute("SELECT * FROM brasil_emissao WHERE rowid <= 5")
print(c.fetchall())
conexao.commit()
conexao.close()

[(0, 1970, 'Agropecuária', 'Cultivo do Arroz', 'Diretas', 'Outros', 'Vegetal', 'Arroz', 'Emissão', 'CH4 (t)', None, None, 230462.17), (1, 1971, 'Agropecuária', 'Cultivo do Arroz', 'Diretas', 'Outros', 'Vegetal', 'Arroz', 'Emissão', 'CH4 (t)', None, None, 226016.3), (2, 1972, 'Agropecuária', 'Cultivo do Arroz', 'Diretas', 'Outros', 'Vegetal', 'Arroz', 'Emissão', 'CH4 (t)', None, None, 220101.2), (3, 1973, 'Agropecuária', 'Cultivo do Arroz', 'Diretas', 'Outros', 'Vegetal', 'Arroz', 'Emissão', 'CH4 (t)', None, None, 214195.56), (4, 1974, 'Agropecuária', 'Cultivo do Arroz', 'Diretas', 'Outros', 'Vegetal', 'Arroz', 'Emissão', 'CH4 (t)', None, None, 186862.84)]
